In [1]:
from copy import deepcopy
import pandas as pd
import numpy as np
import json, gzip, os, re, gensim, sys
import datetime as dt
from collections import Counter
from scipy.stats import spearmanr, pearsonr
import statsmodels.api as sm
from sklearn import metrics
from sklearn.metrics import r2_score, make_scorer, mean_absolute_error as mae, mean_squared_error as mse
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import LeaveOneOut, cross_val_score
from sklearn.base import clone
from sklearn.dummy import DummyRegressor
import clustfunc1
import importlib
from matplotlib import pyplot as plt
from itertools import chain, combinations
from sklearn.feature_selection import RFECV, SelectFpr, f_regression
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from itertools import chain, combinations, product

In [2]:
foo = './heldout_clusteval/'
def procclres(fn, foo = foo):
    #print(fn)
    f = open(foo+fn, 'r')
    ls = [[fn] + l.strip().split('\t') for l in f.readlines()[1:]]
    f.close()
    #ls.sort(key = lambda x: float(x[4]))
    return ls

In [3]:
def getress(wb, foo = './heldout_clusteval/'):
    ress = []
    fns = [fn for fn in os.listdir(foo) if fn.startswith(wb+'_')]
    for fn in fns:
        ress += procclres(fn, foo)
    ress.sort(key = lambda x: float(x[4]))
    return ress

In [142]:
getress('Diener', foo = './heldout_clusteval_noRFE/')

[['Diener_11_50.0_1.0.txt',
  'diener',
  'RFECV',
  'DecisionTreeRegressor',
  '0.17096088435374152',
  '{"criterion": "mae", "max_depth": 2, "max_features": "auto", "max_leaf_nodes": 3, "min_samples_leaf": 2, "min_samples_split": 2, "random_state": 42, "splitter": "best"}'],
 ['Diener_11_500.0_1.0.txt',
  'diener',
  'RFECV',
  'RandomForestRegressor',
  '0.1729280504719536',
  '{"max_depth": 2, "max_features": "auto", "min_samples_leaf": 1, "min_samples_split": 2, "n_estimators": 20, "random_state": 42}'],
 ['Diener_11_500.0_1.0VERYRARE.txt',
  'diener',
  'RFECV',
  'DecisionTreeRegressor',
  '0.17301445578231295',
  '{"criterion": "mae", "max_depth": 2, "max_features": "auto", "max_leaf_nodes": 3, "min_samples_leaf": 2, "min_samples_split": 2, "random_state": 42, "splitter": "best"}'],
 ['Diener_11_500.0_1.0.txt',
  'diener',
  'RFECV',
  'AdaBoostRegressor',
  '0.17348040415413965',
  '{"loss": "linear", "n_estimators": 100, "random_state": 42}'],
 ['Diener_11_50.0_1.0.txt',
  'd

In [143]:
getress('WHO', foo = './heldout_clusteval_noRFE/')

[['WHO_11_500.0_1.0VERYRARE.txt',
  'who',
  'RFECV',
  'RandomForestRegressor',
  '0.1541161515681865',
  '{"max_depth": 3, "max_features": "auto", "min_samples_leaf": 1, "min_samples_split": 2, "n_estimators": 20, "random_state": 42}'],
 ['WHO_11_1.0_0.0VERYRARE.txt',
  'who',
  'RFECV',
  'RandomForestRegressor',
  '0.15424440839005044',
  '{"max_depth": 3, "max_features": "auto", "min_samples_leaf": 1, "min_samples_split": 2, "n_estimators": 20, "random_state": 42}'],
 ['WHO_14_500.0_1.0VERYRARE.txt',
  'who',
  'RFECV',
  'RandomForestRegressor',
  '0.15455674519690255',
  '{"max_depth": 2, "max_features": "auto", "min_samples_leaf": 1, "min_samples_split": 2, "n_estimators": 20, "random_state": 42}'],
 ['WHO_12_1.0_0.0.txt',
  'who',
  'RFECV',
  'RandomForestRegressor',
  '0.1546650971585279',
  '{"max_depth": 2, "max_features": "auto", "min_samples_leaf": 1, "min_samples_split": 2, "n_estimators": 20, "random_state": 42}'],
 ['WHO_12_1.0_0.0VERYRARE.txt',
  'who',
  'RFECV',
  

In [31]:
gfoo = '../gold_split/'
wdf2 = pd.read_csv(gfoo + 'heldout-only-words1year.csv', index_col=0)

freqsdf = pd.read_csv(gfoo + 'heldout-freqs.csv', index_col = 0)
def clusterdf(fn, rare = False):
    cdf = pd.read_csv(fn, index_col = 0)
    resd, centerd = {}, {}
    ids, centers = [], []
    for i, r in cdf.iterrows():
        #if rare or (r.WHO_P < 0.05) or (r.Diener_P < 0.05):
            ws = r.ex.strip().split(', ')
            for w in ws:
                resd[w] = i
            if rare:
                ids.append(i)
                centers.append(np.mean([model[w] for w in ws], axis=0))
                
            #print(i)
    if rare:
        cls = []
        for w in wdf2.columns:
            cl = resd.get(w, -1)
            
            if (cl == -1) and (w in model.vocab) and (w in freqsdf.index) and (freqsdf.loc[w, '0'] >= 150):
                
                dists = [cosine(model[w], center) for center in centers]
                dists = [d if d >= 0. else 1 for d in dists ]
                #cl1 = ids[np.argmin(dists)]
            if (cl != -1) and ((cdf.loc[cl, 'WHO_P'] < 0.05) or (cdf.loc[cl, 'Diener_P'] < 0.05)):
                #cl = cl
                print(w, cl, cdf.loc[cl, 'WHO_P'], cdf.loc[cl, 'Diener_P'])
                cls.append(cl)
            else:
                cls.append(-1)
        print(cls)
    else:
        cls = np.asarray([resd.get(w, -1) for w in wdf2.columns])
    cldf = wdf2.groupby(by = cls, axis = 'columns').sum()
    #print(cldf.shape)
    cldf.drop(-1, axis=1, inplace = True)
    return cldf

    
rat = {'who': 25., 'diener': 30.}
metrs = ['MAE', 'Pearson', 'R-2']
def set_params(func, parameters):
    for parameter, value in parameters.items():
        if hasattr(func, parameter):
            setattr(func, parameter, value)
    return func

In [44]:
def eval_nestedCV(X, y, wb, step=1, verbose=0, method='', fsname='', funcname='Ridge', fout=None, fout_mean=None, cvn=10):
    grids = parameters[funcname]
    print(funcname, flush=True)
    wb = wb.lower()
    sortedarg = np.asarray(y.argsort())
    cv = []
    X = np.asarray(X)
    y = np.asarray(y)
    for ti in range(cvn):
        
            test = sortedarg[ti::cvn]
            train = np.setdiff1d(sortedarg, test)
            cv.append([train, test])
    
    maes, prs, r2s = [], [], []
    
    best_params = {}
        
    bestmae = 1000
    #
    #print(len(train), len(dev), len(test))
    
    for grid in grids:
        keys = sorted(grid.keys())
        for comb in product(*[val for k, val in sorted(grid.items())]):
            print(comb, flush=True)
            devmaes = []
            #for i, [train, test] in enumerate(cv):
            #    print(i, 'CV fold..', flush=True)
            model = set_params(deepcopy(models[funcname]), dict(zip(keys, comb)))
                #lr = model
            #    if verbose:
            #        print('Model:', funcname, 'params:', dict(zip(keys, comb)), 'est:', model)

            fs = RFECV(model, cv=cv, scoring=make_scorer(mae, greater_is_better=False), step=step, verbose=verbose)
                #print(X.shape, y.shape)
            fs.fit(X, y)
            if verbose:
                print(fs.n_features_, flush=True)
            resmae = fs.grid_scores_.max()/-rat[wb]
            if verbose:
                    print(resmae, flush=True)
                    #print('Others:',fs.grid_scores_.mean()/-rat[wb], fs.grid_scores_.min()/-rat[wb])
            devmaes.append(resmae)
            devmae = np.mean(devmaes)
            if devmae < bestmae:
                    bestmae = devmae
                    best_params = dict(zip(keys, comb))
                    best_params['fs'] = fs
                    best_params['fs_sup'] = int(sum(fs.get_support()==1))
                    
    print('Dev: ', bestmae, best_params, flush=True)
    '''est = set_params(deepcopy(models[funcname]), best_params)                                 
    est.fit(best_params['fs'].transform(X[train]), y[train])
    testpred = est.predict(best_params['fs'].transform(X[test]))

    cur_mae = mae(y[test], testpred)/rat[wb]
    maes.append(cur_mae)
    cur_prs = pearsonr(y[test], testpred)[0]
    prs.append(cur_prs)
    cur_r2 = r2_score(y[test], testpred)
    r2s.append(cur_r2)
    '''
    if fout is not None:
                fout.write('{0}\t{1}\t{2}\t{3}\t{4}\n'.format(wb, method, funcname, 
                                                                              bestmae, 
                                                                              json.dumps({k: v for k, v in best_params.items() if k != 'fs'})))
                fout.flush()
    #print('Test: ', [round(np.mean(x), 4) for x in [maes, prs, r2s]], flush=True)
    if fout_mean is not None:
        fout_mean.write('{0}\t{1}\t{2}\t{3}\t{4}\t{5}\t{6}\t{7}\n'.format(method, wb, fsname, funcname,
                                                                     round(np.mean(devmaes), 4),
                                                                     round(np.mean(maes), 4),
                                                                     round(np.mean(prs), 4),
                                                                     round(np.mean(r2s), 4)))
    return fs

In [45]:
models = {'DecisionTreeRegressor': DecisionTreeRegressor(),
          'RandomForestRegressor': RandomForestRegressor(),
          'Ridge': Ridge(),
          'AdaBoostRegressor': AdaBoostRegressor(),
          'LinearRegression': LinearRegression(),
          'Lasso': Lasso(),
          'ElasticNet': ElasticNet()}

In [83]:
parameters = {}
fdiener = 'AdaBoostRegressor'
fwho = 'RandomForestRegressor'
parameters['AdaBoostRegressor'] = [{'loss': ['exponential'], 'n_estimators': [10], 'random_state': [42]}]
parameters['RandomForestRegressor'] = [{'n_estimators': [20], 'max_depth': [3],
                                        'min_samples_split': [2], 'min_samples_leaf': [1], 'max_features': ['auto'],
                                         'random_state': [42]}]

In [62]:
#'Diener_11_500.0_1.0.txt'
wb = 'Diener'
dienerfn = './h-clusters0104/11_cos_500.0_1.0.txt'
cldf = clusterdf(dienerfn)
cldf['DF_' + wb.lower() + '_score'] = wdf2['DF_' + wb.lower() + '_score']
data = {}
data[wb] = cldf.dropna(subset = ['DF_' + wb.lower() + '_score'])
fs = eval_nestedCV(data[wb].drop(['DF_' + wb.lower() + '_score'], axis=1), data[wb]['DF_' + wb.lower() + '_score'], wb, step=1, verbose=1, 
              method='RFECV', fsname='', funcname=fdiener, cvn = 10)

AdaBoostRegressor
('exponential', 10, 42)
Fitting estimator with 146 features.
Fitting estimator with 145 features.
Fitting estimator with 144 features.
Fitting estimator with 143 features.
Fitting estimator with 142 features.
Fitting estimator with 141 features.
Fitting estimator with 140 features.
Fitting estimator with 139 features.
Fitting estimator with 138 features.
Fitting estimator with 137 features.
Fitting estimator with 136 features.
Fitting estimator with 135 features.
Fitting estimator with 134 features.
Fitting estimator with 133 features.
Fitting estimator with 132 features.
Fitting estimator with 131 features.
Fitting estimator with 130 features.
Fitting estimator with 129 features.
Fitting estimator with 128 features.
Fitting estimator with 127 features.
Fitting estimator with 126 features.
Fitting estimator with 125 features.
Fitting estimator with 124 features.
Fitting estimator with 123 features.
Fitting estimator with 122 features.
Fitting estimator with 121 featur

In [146]:
#Diener all: Diener_11_50.0_1.0.txt
wb = 'Diener'
dienerfn = './h-clusters0104/11_cos_50.0_1.0.txt'
cldf = clusterdfgold(dienerfn)
cldf.to_csv(gfoo+'besthall-clusters_Diener.csv')

In [147]:
#WHO all: WHO_11_500.0_1.0VERYRARE.txt
wb = 'WHO'
dienerfn = './h-clusters0104/11_cos_500.0_1.0VERYRARE.txt'
cldf = clusterdfgold(dienerfn)
cldf.to_csv(gfoo+'besthall-clusters_WHO.csv')
cldf

,,1,2,3,4,5,6,9,10,16,19,...,596,607,614,630,631,633,638,651,665,695
vk_id_encrypted,DF_freud_install_time,,,,,,,,,,,,,,,,,,,,,
137479231,2018-08-20 10:43:15,0.310291,0.000154,0.003244,0.000474,0.000072,0.000391,0.001947,0.000216,0.000484,0.000762,...,0.000124,0.000144,0.000103,0.000113,0.000021,0.000031,0.000257,0.000185,0.000371,0.041454
180350141,2018-10-18 17:07:07,0.291562,0.000113,0.001616,0.000430,0.000219,0.000384,0.000457,0.000788,0.000179,0.001066,...,0.000073,0.000086,0.000079,0.000119,0.000033,0.000093,0.000093,0.000060,0.000358,0.042935
190703550,2018-09-14 12:38:34,0.253743,0.000165,0.003113,0.000638,0.000205,0.000307,0.002569,0.000134,0.000150,0.001206,...,0.000102,0.000158,0.000102,0.000118,0.000173,0.000087,0.000158,0.000095,0.000386,0.044963
202633826,2018-09-23 02:38:48,0.292900,0.000172,0.001518,0.000276,0.000103,0.000172,0.001483,0.000310,0.000069,0.002139,...,0.000069,0.000034,0.000103,0.000034,0.000000,0.000034,0.000172,0.000069,0.000138,0.040813
333101115,2018-05-12 10:21:30,0.346461,0.000410,0.002440,0.000410,0.000455,0.000510,0.003915,0.000166,0.000466,0.001675,...,0.000078,0.000067,0.000100,0.000122,0.000111,0.000166,0.000399,0.000144,0.000266,0.055219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693337702,2018-05-12 01:13:30,0.332011,0.000115,0.001976,0.000331,0.000027,0.000277,0.001646,0.000054,0.000229,0.001369,...,0.000121,0.000027,0.000074,0.000148,0.000040,0.000074,0.000175,0.000115,0.000499,0.049227
220279175,2018-08-25 01:44:05,0.331522,0.000092,0.001240,0.000291,0.000131,0.001041,0.000321,0.000137,0.000146,0.000940,...,0.000128,0.000036,0.000059,0.000074,0.000059,0.000059,0.000042,0.000048,0.000294,0.045050
568464126,2018-08-14 02:36:17,0.340674,0.000168,0.002150,0.000274,0.000162,0.000347,0.001433,0.000045,0.000291,0.001226,...,0.000067,0.000101,0.000207,0.000118,0.000022,0.000202,0.000140,0.000056,0.000263,0.052648


In [84]:
#'WHO_11_1.0_0.0VERYRARE.txt'
wb = 'WHO'
whofn = './h-clusters0104/11_cos_1.0_0.0VERYRARE.txt'
cldf = clusterdf(whofn)
cldf['DF_' + wb.lower() + '_score'] = wdf2['DF_' + wb.lower() + '_score']
data[wb] = cldf.dropna(subset = ['DF_' + wb.lower() + '_score'])
fs = eval_nestedCV(data[wb].drop(['DF_' + wb.lower() + '_score'], axis=1), data[wb]['DF_' + wb.lower() + '_score'], wb, step=1, verbose=1, 
              method='RFECV', fsname='', funcname=fwho, cvn = 10)

RandomForestRegressor
(3, 'auto', 1, 2, 20, 42)
Fitting estimator with 134 features.
Fitting estimator with 133 features.
Fitting estimator with 132 features.
Fitting estimator with 131 features.
Fitting estimator with 130 features.
Fitting estimator with 129 features.
Fitting estimator with 128 features.
Fitting estimator with 127 features.
Fitting estimator with 126 features.
Fitting estimator with 125 features.
Fitting estimator with 124 features.
Fitting estimator with 123 features.
Fitting estimator with 122 features.
Fitting estimator with 121 features.
Fitting estimator with 120 features.
Fitting estimator with 119 features.
Fitting estimator with 118 features.
Fitting estimator with 117 features.
Fitting estimator with 116 features.
Fitting estimator with 115 features.
Fitting estimator with 114 features.
Fitting estimator with 113 features.
Fitting estimator with 112 features.
Fitting estimator with 111 features.
Fitting estimator with 110 features.
Fitting estimator with 109 

In [85]:
len(fs.get_support()), sum(fs.get_support())

(134, 19)

In [86]:
cols = data[wb].drop(['DF_' + wb.lower() + '_score'], axis=1).columns[fs.get_support()]
cols

Index([  1,  12,  14,  22,  50,  68,  85, 120, 132, 200, 334, 372, 374, 381,
       465, 495, 567, 610, 639],
      dtype='object')

In [63]:
allwdf = pd.read_csv('word-matrix-1year_mindf10_minwords20.csv', index_col=0)
g1 = pd.read_csv(gfoo+'traincorr1.csv', index_col=[0, 1])
g2 = pd.read_csv(gfoo+'testcorr1.csv', index_col=[0,1])
g = pd.concat([g1, g2])
allwdf = g.join(allwdf)[allwdf.columns]

In [67]:
def clusterdfgold(fn):
    cdf = pd.read_csv(fn, index_col = 0)
    resd, centerd = {}, {}
    ids, centers = [], []
    for i, r in cdf.iterrows():
        #if rare or (r.WHO_P < 0.05) or (r.Diener_P < 0.05):
            ws = r.ex.strip().split(', ')
            for w in ws:
                resd[w] = i
            #if rare:
            #    ids.append(i)
            #    centers.append(np.mean([model[w] for w in ws], axis=0))
                
            #print(i)
    cls = np.asarray([resd.get(w, -1) for w in allwdf.columns])
    cldf = allwdf.groupby(by = cls, axis = 'columns').sum()
    #print(cldf.shape)
    cldf.drop(-1, axis=1, inplace = True)
    return cldf

In [68]:
dienercldf = clusterdfgold(dienerfn)
dienercldf

,,1,2,3,4,5,6,9,10,16,19,...,596,607,614,630,631,633,638,651,665,695
vk_id_encrypted,DF_freud_install_time,,,,,,,,,,,,,,,,,,,,,
137479231,2018-08-20 10:43:15,0.304482,0.000031,0.002863,0.000185,0.000000,0.000124,0.001936,0.000000,0.000041,0.000474,...,0.000103,0.000021,0.000010,0.000103,0.000010,0.000000,0.000000,0.000082,0.000299,0.037427
180350141,2018-10-18 17:07:07,0.286867,0.000033,0.001470,0.000119,0.000113,0.000212,0.000238,0.000179,0.000013,0.000662,...,0.000046,0.000020,0.000013,0.000093,0.000033,0.000026,0.000007,0.000033,0.000344,0.040352
190703550,2018-09-14 12:38:34,0.248195,0.000008,0.002900,0.000173,0.000047,0.000095,0.002151,0.000008,0.000008,0.000764,...,0.000071,0.000008,0.000000,0.000118,0.000126,0.000047,0.000008,0.000016,0.000355,0.041354
202633826,2018-09-23 02:38:48,0.287346,0.000172,0.001276,0.000207,0.000069,0.000000,0.001276,0.000103,0.000069,0.001587,...,0.000069,0.000000,0.000000,0.000034,0.000000,0.000034,0.000000,0.000069,0.000138,0.039847
333101115,2018-05-12 10:21:30,0.340361,0.000089,0.002096,0.000133,0.000233,0.000189,0.003893,0.000067,0.000033,0.001065,...,0.000055,0.000022,0.000011,0.000122,0.000100,0.000000,0.000100,0.000055,0.000211,0.051548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693337702,2018-05-12 01:13:30,0.328504,0.000054,0.001895,0.000094,0.000013,0.000067,0.001632,0.000007,0.000013,0.001025,...,0.000088,0.000027,0.000013,0.000148,0.000040,0.000013,0.000047,0.000000,0.000499,0.047629
220279175,2018-08-25 01:44:05,0.328664,0.000027,0.001127,0.000077,0.000039,0.000874,0.000306,0.000021,0.000009,0.000809,...,0.000125,0.000018,0.000009,0.000074,0.000051,0.000012,0.000012,0.000015,0.000271,0.043399
568464126,2018-08-14 02:36:17,0.334158,0.000022,0.001858,0.000101,0.000084,0.000213,0.001371,0.000000,0.000011,0.000929,...,0.000050,0.000006,0.000028,0.000112,0.000017,0.000050,0.000045,0.000017,0.000218,0.050756


In [87]:
whocldf = clusterdfgold(whofn)
whocldf

,,1,6,7,11,12,14,17,18,22,28,...,584,593,607,609,610,616,639,647,659,667
vk_id_encrypted,DF_freud_install_time,,,,,,,,,,,,,,,,,,,,,
137479231,2018-08-20 10:43:15,0.316470,0.000278,0.000731,0.000268,0.000711,0.000031,0.000113,0.000165,0.000072,0.000175,...,0.000165,0.000185,0.000113,0.000824,0.000031,0.000144,0.000062,0.000309,0.000566,0.038055
180350141,2018-10-18 17:07:07,0.299827,0.000450,0.001748,0.000298,0.000132,0.000106,0.000146,0.000046,0.000199,0.000060,...,0.000079,0.000066,0.000119,0.000437,0.000046,0.000093,0.000079,0.000199,0.000126,0.033756
190703550,2018-09-14 12:38:34,0.267525,0.000150,0.000607,0.000244,0.000158,0.000087,0.000465,0.000087,0.000772,0.000489,...,0.000158,0.000095,0.000118,0.000512,0.000213,0.000110,0.000189,0.000276,0.000158,0.038068
202633826,2018-09-23 02:38:48,0.311564,0.000138,0.000276,0.000138,0.000207,0.000138,0.000345,0.000000,0.000586,0.000034,...,0.000000,0.000241,0.000034,0.000759,0.000000,0.000000,0.000103,0.000103,0.000172,0.033395
333101115,2018-05-12 10:21:30,0.363052,0.000166,0.000599,0.000466,0.000399,0.000244,0.000200,0.000166,0.000067,0.000111,...,0.000089,0.000233,0.000122,0.000876,0.000144,0.000355,0.000266,0.000311,0.000388,0.043662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693337702,2018-05-12 01:13:30,0.343101,0.000108,0.000391,0.000074,0.000101,0.000155,0.000135,0.000054,0.000121,0.000074,...,0.000027,0.000128,0.000148,0.000337,0.000040,0.000155,0.000054,0.000121,0.000236,0.038914
220279175,2018-08-25 01:44:05,0.342663,0.000134,0.000681,0.000155,0.000125,0.000042,0.000083,0.000036,0.000175,0.000089,...,0.000036,0.000095,0.000074,0.000357,0.000101,0.000033,0.000092,0.000125,0.000065,0.038617
568464126,2018-08-14 02:36:17,0.354154,0.000078,0.000297,0.000291,0.000140,0.000146,0.000162,0.000062,0.000162,0.000123,...,0.000084,0.000162,0.000112,0.000476,0.000034,0.000151,0.000560,0.000157,0.000297,0.041480


In [70]:
dienercldf[cols].to_csv(gfoo+'besth-clusters_Diener.csv')

In [88]:
whocldf[cols].to_csv(gfoo+'besth-clusters_WHO.csv')

In [89]:
whocldf[cols]

,,1,12,14,22,50,68,85,120,132,200,334,372,374,381,465,495,567,610,639
vk_id_encrypted,DF_freud_install_time,,,,,,,,,,,,,,,,,,,
137479231,2018-08-20 10:43:15,0.316470,0.000711,0.000031,0.000072,0.000082,0.001071,0.000165,0.000257,0.000721,0.000762,0.000278,0.000484,0.000330,0.000288,0.000165,0.000134,0.000134,0.000031,0.000062
180350141,2018-10-18 17:07:07,0.299827,0.000132,0.000106,0.000199,0.000060,0.000722,0.000146,0.000086,0.000742,0.001298,0.000086,0.000245,0.000159,0.000404,0.000132,0.000040,0.000119,0.000046,0.000079
190703550,2018-09-14 12:38:34,0.267525,0.000158,0.000087,0.000772,0.000079,0.000441,0.000110,0.000189,0.000859,0.000796,0.000165,0.000559,0.000213,0.000213,0.000197,0.000063,0.000173,0.000213,0.000189
202633826,2018-09-23 02:38:48,0.311564,0.000207,0.000138,0.000586,0.000000,0.000517,0.000069,0.000103,0.000345,0.000310,0.000034,0.000069,0.000103,0.000172,0.000207,0.000000,0.000069,0.000000,0.000103
333101115,2018-05-12 10:21:30,0.363052,0.000399,0.000244,0.000067,0.000055,0.000455,0.000189,0.000377,0.000665,0.000887,0.000166,0.000233,0.000311,0.000266,0.000299,0.000111,0.000155,0.000144,0.000266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693337702,2018-05-12 01:13:30,0.343101,0.000101,0.000155,0.000121,0.000061,0.000533,0.000074,0.000175,0.000648,0.000614,0.000040,0.000155,0.000067,0.000007,0.000074,0.000067,0.000175,0.000040,0.000054
220279175,2018-08-25 01:44:05,0.342663,0.000125,0.000042,0.000175,0.000042,0.000491,0.000074,0.000244,0.000434,0.000467,0.000042,0.000202,0.000089,0.000059,0.000098,0.000054,0.000068,0.000101,0.000092
568464126,2018-08-14 02:36:17,0.354154,0.000140,0.000146,0.000162,0.000050,0.000672,0.000067,0.000313,0.000403,0.000655,0.000084,0.000174,0.000168,0.000202,0.000509,0.000039,0.000090,0.000034,0.000560


# Evaluation

In [2]:
def vieweval(wb, rfoo = './res_april2021/', thresh = 0.33):
    i = 0
    resd = {}
    for fn in os.listdir(rfoo):
        if fn.startswith('final') and ('testscore' in fn) and (wb in fn):
            #print(fn)
            f = open(rfoo+fn, 'r')
            for line in f:
                if line.startswith('method'):
                    continue
                line = line.strip()
                l = line.split('\t')
                resd[l[2]] = resd.get(l[2], [])
                resd[l[2]].append(l + [fn])
                #print(l[-1])
                if float(l[-2]) > thresh:
                    i+=1
                    print(str(i)+'.', fn, line.strip())
            f.close()
    for k,val in resd.items():
        resd[k].sort(key = lambda x: float(x[-4]))
        print(k, len(val))
        for v in val:
            print(' '.join(v))
        print('')

In [3]:
vieweval('WHO')

1. final_reg4_testscores_max_Lasso_WHO_RFECV.tsv RFECV	who	AppCats + Behavior + Words	Lasso	0.1369	0.1464	0.3636	0.074
2. final_reg4_testscores_max_ElasticNet_WHO_RFECV.tsv RFECV	who	Words	ElasticNet	0.1382	0.1456	0.354	0.0864
3. final_reg4_testscores_max_ElasticNet_WHO_RFECV.tsv RFECV	who	AppCats + Words	ElasticNet	0.1345	0.1456	0.3581	0.0995
4. final_reg4_testscores_max_ElasticNet_WHO_RFECV.tsv RFECV	who	RuLIWC + Words	ElasticNet	0.1394	0.1438	0.3729	0.1058
5. final_reg4_testscores_max_ElasticNet_WHO_RFECV.tsv RFECV	who	AppCats + Behavior + Words	ElasticNet	0.1326	0.1467	0.3316	0.0731
6. final_reg4_testscores_max_ElasticNet_WHO_RFECV.tsv RFECV	who	AppCats + RuLIWC + Behavior + Words	ElasticNet	0.1325	0.1461	0.3592	0.0999
7. final_clust_reg4_testscores_max_Lasso_WHO_RFECV.tsv RFECV	who	clusters + AppCats + Behavior + Words	Lasso	0.1373	0.1457	0.3339	0.0701
8. final_clust_reg4_testscores_max_ElasticNet_WHO_RFECV.tsv RFECV	who	clusters + Words	ElasticNet	0.1394	0.146	0.3488	0.0954
9. fi

In [4]:
vieweval('Diener')

1. final_reg4_testscores_max_ElasticNet_Diener_RFECV.tsv RFECV	diener	AppCats + Behavior + Words	ElasticNet	0.1628	0.1746	0.3688	0.0848
2. final_reg4_testscores_max_ElasticNet_Diener_RFECV.tsv RFECV	diener	RuLIWC + Behavior + Words	ElasticNet	0.1667	0.1767	0.3387	0.0769
3. finalNEW_clust_reg4_testscores_max_ElasticNet_Diener_RFECV.tsv RFECV	diener	Words	ElasticNet	0.1706	0.1744	0.3402	0.1022
4. finalNEW_clust_reg4_testscores_max_ElasticNet_Diener_RFECV.tsv RFECV	diener	clusters + Words	ElasticNet	0.1714	0.1716	0.3627	0.0901
5. finalNEW_clust_reg4_testscores_max_ElasticNet_Diener_RFECV.tsv RFECV	diener	RuLIWC + Words	ElasticNet	0.1699	0.1722	0.352	0.0988
6. finalNEW_clust_reg4_testscores_max_ElasticNet_Diener_RFECV.tsv RFECV	diener	clusters + RuLIWC + Words	ElasticNet	0.1712	0.1712	0.3673	0.1192
7. finalNEW_clust_reg4_testscores_max_ElasticNet_Diener_RFECV.tsv RFECV	diener	clusters + Behavior + Words	ElasticNet	0.1695	0.1712	0.3459	0.1228
8. finalNEW_clust_reg4_testscores_max_ElasticNet

In [239]:
vieweval('WHO', './res_april2021_noRFE/', thresh=0.3)

1. final_clust_reg4_testscores_max_AdaBoostRegressor_WHO_RFECV.tsv RFECV	who	Words	AdaBoostRegressor	0.1402	0.1472	0.3021	0.054
2. final_clust_reg4_testscores_max_AdaBoostRegressor_WHO_RFECV.tsv RFECV	who	RuLIWC + Behavior + Words	AdaBoostRegressor	0.1425	0.1461	0.3148	0.0679
clusters 7
RFECV who clusters RandomForestRegressor 0.1511 0.1553 0.065 -0.0339
AppCats 7
RFECV who AppCats RandomForestRegressor 0.1489 0.1534 0.0675 -0.0223
RuLIWC 7
RFECV who RuLIWC ElasticNet 0.1507 0.1524 0.1831 0.0279
Behavior 7
RFECV who Behavior Ridge 0.1477 0.1525 0.1522 -0.0042
Words 7
RFECV who Words AdaBoostRegressor 0.1402 0.1472 0.3021 0.054
clusters + AppCats 7
RFECV who clusters + AppCats RandomForestRegressor 0.1496 0.1561 0.0336 -0.0617
clusters + RuLIWC 7
RFECV who clusters + RuLIWC Lasso 0.1491 0.1546 0.0652 -0.0048
clusters + Behavior 7
RFECV who clusters + Behavior RandomForestRegressor 0.1506 0.1522 0.1365 -0.0028
clusters + Words 7
RFECV who clusters + Words ElasticNet 0.148 0.1516 0.1991 0

In [240]:
vieweval('Diener', './res_april2021_noRFE/', thresh=0.2)

clusters 7
RFECV diener clusters Ridge 0.1803 0.1862 0.0108 -0.017
AppCats 7
RFECV diener AppCats Lasso 0.1839 0.1874 0.0074 -0.0207
RuLIWC 7
RFECV diener RuLIWC Lasso 0.1809 0.1824 0.1392 0.0243
Behavior 7
RFECV diener Behavior DecisionTreeRegressor 0.1806 0.1806 0.144 -0.0098
Words 7
RFECV diener Words Ridge 0.1785 0.1829 0.1285 0.0202
clusters + AppCats 7
RFECV diener clusters + AppCats Lasso 0.1848 0.1859 -0.0065 -0.0098
clusters + RuLIWC 7
RFECV diener clusters + RuLIWC ElasticNet 0.1794 0.1832 0.1246 0.0012
clusters + Behavior 7
RFECV diener clusters + Behavior DecisionTreeRegressor 0.1847 0.1847 0.1354 -0.0262
clusters + Words 7
RFECV diener clusters + Words Ridge 0.1787 0.1846 0.0578 -0.0051
AppCats + RuLIWC 7
RFECV diener AppCats + RuLIWC AdaBoostRegressor 0.1808 0.1888 0.0601 -0.064
AppCats + Behavior 7
RFECV diener AppCats + Behavior Ridge 0.1804 0.1866 0.0741 -0.0337
AppCats + Words 7
RFECV diener AppCats + Words Ridge 0.1803 0.1833 0.1562 0.0169
RuLIWC + Behavior 7
RFECV d

In [211]:
rfoo = './res_april2021_noRFE/'
i = 0
for fn in os.listdir(rfoo):
    if 'testscore' in fn:
        #print(fn)
        f = open(rfoo+fn, 'r')
        for line in f:
            if line.startswith('method'):
                continue
            l = line.strip().split('\t')
            #print(l[-1])
            if float(l[-2]) > 0.3:
                i+=1
                print(str(i)+'.', fn, line.strip())
        f.close()

1. final_clust_reg4_testscores_max_AdaBoostRegressor_WHO_RFECV.tsv RFECV	who	Words	AdaBoostRegressor	0.1402	0.1472	0.3021	0.054
2. final_clust_reg4_testscores_max_AdaBoostRegressor_WHO_RFECV.tsv RFECV	who	RuLIWC + Behavior + Words	AdaBoostRegressor	0.1425	0.1461	0.3148	0.0679
